# Direct and inverse geometry of 3d robots
This notebook introduces the kinematic tree of Pinocchio for a serial manipulator, explain how to compute the forward and inverse geometry (from configuration to end-effector placements, and inversely). The ideas are examplified with a simplified case-study taken from parallel robotics.

In [1]:
import magic_donotload  # noqa: F401

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Set up
We will need Pinocchio, Gepetto-Viewer, SciPy for the solvers

In [2]:
import time
import math
import numpy as np
from numpy.linalg import norm
import pinocchio as pin
import example_robot_data as robex
from scipy.optimize import fmin_bfgs

## Kinematic tree in Pinocchio
Let's now play with 3D robots. We will load the models from URDF files.

*The robot UR5* is a low-cost manipulator robot with good performances. It is a fixed robot with one 6-DOF arms developed by the Danish company Universal Robot. All its 6 joints are revolute joints. Its configuration is in R^6 and is not subject to any constraint. The model of UR5 is described in a URDF file, with the visuals of the bodies of the robot being described as meshed (i.e. polygon soups) using the Collada format ".dae". Both the URDF and the DAE files are available in the repository in the model directory. 

This robot model, as well as other models used in the notebooks, are installed from the apt paquet robotpkg-example-robot-data and stored in /opt/openrobots/share/example-robot-data.


In [3]:
robot = robex.load('ur5')

The kinematic tree is represented by two C++ objects called Model (which contains the model constants: lengths, masses, names, etc) and Data (which contains the working memory used by the model algorithms). Both C\++ objects are contained in a unique Python class. The first class is called RobotWrapper and is generic.

In [4]:
print(robot.model)

Nb joints = 7 (nq=6,nv=6)
  Joint 0 universe: parent=0
  Joint 1 shoulder_pan_joint: parent=0
  Joint 2 shoulder_lift_joint: parent=1
  Joint 3 elbow_joint: parent=2
  Joint 4 wrist_1_joint: parent=3
  Joint 5 wrist_2_joint: parent=4
  Joint 6 wrist_3_joint: parent=5



For the next steps, we are going to work with the RobotWrapper.

Import the class RobotWrapper and create an instance of this class in the python terminal. At initialization, RobotWrapper will read the model description in the URDF file given as argument. In the following, we will use the model of the UR5 robot, available in the directory "models" of pinocchio (available in the homedir of the VBox). The code of the RobotWrapper class is in /opt/openrobots/lib/python2.7/site-packages/pinocchio/robot_wrapper.py . Do not hesitate to have a look at it and to take inspiration from the implementation of the class functions.

Here are some import methods of the class.


* robot.q0 contains a reference initial configuration of the robot (not a pretty good one for the UR-5).

* robot.index('joint name') returns the index of the joint.

In [5]:
robot.index(' wrist_3_joint')

7

* robot.model.names is a container (~list) that contains all the joint names

In [6]:
for i, n in enumerate(robot.model.names):
    print(i, n)

0 universe
1 shoulder_pan_joint
2 shoulder_lift_joint
3 elbow_joint
4 wrist_1_joint
5 wrist_2_joint
6 wrist_3_joint


* robot.model.frames contains all the import frames attached to the robot. 

In [7]:
for f in robot.model.frames:
    print(f.name, 'attached to joint #', f.parent)

universe attached to joint # 0
root_joint attached to joint # 0
world attached to joint # 0
world_joint attached to joint # 0
base_link attached to joint # 0
base_link-base_fixed_joint attached to joint # 0
base attached to joint # 0
shoulder_pan_joint attached to joint # 1
shoulder_link attached to joint # 1
shoulder_lift_joint attached to joint # 2
upper_arm_link attached to joint # 2
elbow_joint attached to joint # 3
forearm_link attached to joint # 3
wrist_1_joint attached to joint # 4
wrist_1_link attached to joint # 4
wrist_2_joint attached to joint # 5
wrist_2_link attached to joint # 5
wrist_3_joint attached to joint # 6
wrist_3_link attached to joint # 6
ee_fixed_joint attached to joint # 6
ee_link attached to joint # 6
wrist_3_link-tool0_fixed_joint attached to joint # 6
tool0 attached to joint # 6


robot.placement(idx) and robot.framePlacement(idx) returns the placement (i.e. translation+rotation of the joint / frame in argument.

In [8]:
a = robot.placement(robot.q0, 6)  # Placement of the end effector joint.
b = robot.framePlacement(robot.q0, 22)  # Placement of the end effector tip.

tool_axis = b.rotation[:, 2]  # Axis of the tool

The dimension of the configuration space (i.e. the number of joints) is given in:

In [9]:
NQ = robot.model.nq
NV = robot.model.nv  # for this simple robot, NV == NQ

## Display simple geometries
The robot is displayed in the viewer. We are going to use Meshcat to visualize the 3d robot and scene. First open the viewer and load the robot geometries.

In [10]:
from utils.meshcat_viewer_wrapper import MeshcatVisualizer, colors  # noqa: E402

viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


In [11]:
viz.viewer.jupyter_cell()

A configuration *q* can be displayed in the viewer:

In [12]:
q = np.array([-1., -1.5, 2.1, -.5, -.5, 0])

viz.display(q)

Other geometries (cubes, spheres, etc) can be displayed as well.

In [13]:
# Add a red box in the viewer
ballID = "world/ball"
radius = 0.1
viz.addSphere(ballID, radius, colors.red)

# Place the ball at the position ( 0.5, 0.1, 0.2 )
# The viewer expect position and rotation, apppend the identity quaternion
ball_position = [0.5, 0.1, 0.2]
q_ball = ball_position +  [1, 0, 0, 0]
viz.applyConfiguration(ballID, q_ball)


# Pick and place 3D

### Inverse geometry in 3D for pick position...

We will use inverse geometry method to find a configuration of the robot so that the end effector touches the ball. We will use an unconstrained optimization method.

We can use the  the scipy solver [used in the previous notebook](1_geometry_2d.ipynb#section_optim), compute a configuration q where the end effector reaches p.

For that, implement a cost function that takes a configuration as argument and returns the squared distance between the end effetor tip and the target accounting for the fact we want to touch the ball. Here you can ignore the collision.

In [14]:
target = np.array(ball_position)  # x,y,z

In [15]:
q_touch = robot.q0  # REPLACE WITH INVERSE GEOMETRY PROCEDURE

Here is a solution if you need it

In [16]:
# %load tp2/generated/invgeom3d_1
def cost(q):
    '''Compute score from a configuration'''
    m = robot.framePlacement(q, 22)
    p = m.translation
    offset = m.rotation[:, 2] * radius
    return norm(p +  offset - target)**2


def callback(q):
    viz.display(q)
    time.sleep(1e-2)

q_touch = fmin_bfgs(cost, robot.q0, callback=callback)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 29
         Function evaluations: 224
         Gradient evaluations: 32


In [17]:
# Take care to explicitely mention copy when you want a copy of array.
q = q_touch.copy()

### ...and simulation of place movement

At the reference position you built, the end effector placement can be obtained by calling 

In [18]:
robot.placement(q, 6).translation

array([ 0.56595833, -0.00816157,  0.06891268])

Only the translation part of the placement has been selected. The rotation is free.

Now, choose any trajectory you want in the configuration space (it can be sinus-cosinus waves, polynomials, splines, straight lines). Make a for loop to display the robot at sampling positions along this trajectory. The function sleep can be used to slow down the loop.

At each instant of your loop, recompute the position of the ball and display it so that it always "sticks" to the robot end effector, by modifying the template code below:

In [19]:
q = q_touch.copy()
vq = np.array([2., 0, 0, 4., 0, 0])
idx = 6

# TODO: Compute the placement of the ball in the end effector frame
eff_eff_ball = np.zeros(3)  # Position of ball wrt eff in local coordinate

for i in range(200):
    # Chose new configuration of the robot
    q += vq / 40
    q[2] = 1.71 + math.sin(i * 0.05) / 2

    # TODO: Compute the new placement of the ball
    o_ball = np.zeros(3)

    # Display new configuration for robot and ball
    viz.applyConfiguration(ballID, o_ball.tolist() + [1, 0, 0, 0])
    viz.display(q)
    time.sleep(1e-2)

The solution is below, should you need it.

In [20]:
# %load tp2/generated/simple_pick_and_place_4
q = q_touch.copy()
vq = np.array([2., 0, 0, 4., 0, 0])
idx = 6

p = robot.placement(q, idx)
o_eff = p.translation  # Position of end-eff wrt world at current configuration
o_ball = q_ball[:3]  # Position of ball wrt world
o_eff_ball = o_ball - o_eff  # Position of ball wrt eff in world coordinate
eff_eff_ball = p.rotation.T @ o_eff_ball  # Position of ball wrt eff in local coordinate

for i in range(200):
    # Chose new configuration of the robot
    q += vq / 40
    q[2] = 1.71 + math.sin(i * 0.05) / 2

    # Gets the new position of the ball
    o_ball = robot.placement(q, idx) * eff_eff_ball

    # Display new configuration for robot and ball
    viz.applyConfiguration(ballID, o_ball.tolist() + [1, 0, 0, 0])
    viz.display(q)
    time.sleep(1e-2)


# Pick and place 6D

Say now that the object is a rectangle and not a sphere. Pick the object at a reference position with the rotation that is imposed, so that the end effector is aligned with one of the faces of the rectangle.


In [21]:
# Add a red box in the viewer
boxID = "world/box"
try:
    viz.delete(ballID)
except:
    pass
viz.addBox(boxID, [0.1, 0.2, 0.1], colors.magenta)

# Place the box at the position (0.5, 0.1, 0.2)
q_box = [0.5, 0.1, 0.2, 1, 0, 0, 0]
viz.applyConfiguration(boxID, q_box)


Redo the same two questions as before, but now also choosing the orientation of the box. For that, at each robot configuration in your for-loop, compute the box placement wrt the world (let's denote it by oMbox) and display both the box and the robot configuration in the view.

### Inverse geometry in 6D
6D means: translation and rotation. Change the previous cost function for a cost measuring the difference between the current placement root.placement(q,6) and a reference placement oMdes. 
For that, you can use the SE(3) log function to score the distance between two placements. The log returns a 6D velocity, represented by a class Motion, that must be transformed to a vector of R^6 from which you can take the norm.


In [22]:
oMtarget = pin.SE3(pin.utils.rotate('x', 3.14 / 4), np.array([-0.5, 0.1, 0.2]))  # x,y,z
Moffset = pin.XYZQUATToSE3([0., 0., 0.05, 1, 0, 0, 0])  # Offset to make the tip touch the edge
oMbox = oMtarget * Moffset

In [23]:
# TODO: place the box in target

qopt = robot.q0  # Replace with optimization procedure

print('The robot finally reached effector placement at\n', robot.placement(qopt, 6))

The robot finally reached effector placement at
   R =
          -1            0  9.79328e-12
           0            1            0
-9.79328e-12            0           -1
  p =   0.81725   0.10915 -0.005491



The solution if you need it

In [24]:
# %load tp2/generated/invgeom6d_1
viz.applyConfiguration(boxID, oMbox)

def cost(q):
    '''Compute score from a configuration'''
    M = robot.framePlacement(q, 22)
    return norm(pin.log(M.inverse() * oMtarget).vector)


def callback(q):
    viz.display(q)
    time.sleep(1e-2)


qopt = fmin_bfgs(cost, robot.q0, callback=callback)

print('The robot finally reached effector placement at\n', robot.placement(qopt, 6))

         Current function value: 0.000000
         Iterations: 61
         Function evaluations: 831
         Gradient evaluations: 117
The robot finally reached effector placement at
   R =
           1 -1.72203e-09 -9.34074e-09
 -7.8247e-09    -0.706825    -0.707388
-5.38413e-09     0.707388    -0.706825
  p =     -0.5 0.158172 0.141782



Now move the box following the motion.

In [25]:
q = qopt.copy()
vq = np.array([2., 0, 0, 4., 0, 0])
idx = 6

for i in range(100):
    # Chose new configuration of the robot
    q += vq / 40
    q[2] = 1.71 + math.sin(i * 0.05) / 2

    # TODO: replace with good calculation
    oMbox = oMbox

    # Display new configuration for robot and box
    viz.applyConfiguration(boxID, oMbox)
    viz.display(q)
    time.sleep(1e-2)


In [26]:
# %load tp2/generated/simple_pick_and_place_7
q = qopt.copy()
vq = np.array([2., 0, 0, 4., 0, 0])
idx = 6

oMeff = robot.placement(q, idx)  # Placement of end-eff wrt world at current configuration
effMbox = oMeff.inverse() * oMbox  # Placement of box     wrt eff

for i in range(100):
    # Chose new configuration of the robot
    q += vq / 40
    q[2] = 1.71 + math.sin(i * 0.05) / 2

    # Gets the new position of the box
    oMbox = robot.placement(q, idx) * effMbox

    # Display new configuration for robot and box
    viz.applyConfiguration(boxID, oMbox)
    viz.display(q)
    time.sleep(1e-2)


## On inverse geometry while taking collisions into account

As we have seen in the first TP, we can use constrain optimization solver and create a constraint related to the distance between object being positive. Under the hood, it use the distance computation and finite difference pattern to estimate the gradient and use it in the optimization procedure. If it worked well with the UR5 constrained to a plan with 2 degree of freedom, in the general case those approach get stuck. Recent advances on perturbed optimization allows to have better gradient for the collision function and nicely use it in optimization routine. [random smooth collision gradient](https://hal.archives-ouvertes.fr/hal-03780482/file/Differentiable_collision_detection.pdf)

# Optimizing in the quaternion space

Let's now work with a floating robot: the quadruped ANYmal. This robot has 12 joints, but Q-space of size 19 (robot.model.nq) and Q-tangent space of size 18 (robot.model.nv). This is because with need 7D vector to encode the robot placement in space, which indeed to only 6 DOF.


In [27]:
robot = robex.load('solo12')
viz = MeshcatVisualizer(robot)
viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/


In [28]:
viz.display(robot.q0)


Run the following code. Can you explain what just happened? Then correct it to have a proper optimization of ANYmal configuration.

In [29]:

robot.feetIndexes = [robot.model.getFrameId(frameName) for frameName in ['HR_FOOT', 'HL_FOOT', 'FR_FOOT', 'FL_FOOT']]

# --- Add box to represent target
colors = ['red', 'blue', 'green', 'magenta']
for color in colors:
    viz.addSphere("world/%s" % color, .05, color)
    viz.addSphere("world/%s_des" % color, .05, color)

#
# OPTIM 6D #########################################################
#

targets = [
    np.array([-0.7, -0.2, 1.2]),
    np.array([-0.3, 0.5, 0.8]),
    np.array([0.3, 0.1, -0.1]),
    np.array([0.9, 0.9, 0.5])
]
for i in range(4):
    targets[i][2] += 1


def cost(q):
    '''Compute score from a configuration'''
    cost = 0.
    for i in range(4):
        p_i = robot.framePlacement(q, robot.feetIndexes[i]).translation
        cost += norm(p_i - targets[i])**2
    return cost


def callback(q):
    viz.applyConfiguration('world/box', Mtarget)

    for i in range(4):
        p_i = robot.framePlacement(q, robot.feetIndexes[i])
        viz.applyConfiguration('world/%s' % colors[i], p_i)
        viz.applyConfiguration('world/%s_des' % colors[i], list(targets[i]) + [1, 0, 0, 0])

    viz.display(q)
    time.sleep(1e-2)


Mtarget = pin.SE3(pin.utils.rotate('x', 3.14 / 4), np.array([0.5, 0.1, 0.2]))  # x,y,z
qopt = fmin_bfgs(cost, robot.q0, callback=callback)


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 46
         Function evaluations: 1060
         Gradient evaluations: 53


## Configuration of parallel robots
A parallel robot is composed of several kinematic chains (called the robot legs) that are all attached to the same end effector. This imposes strict constraints in the configuration space of the robot: a configuration is valide iff all the legs meets the same end-effector placement. We consider here only the geometry aspect of parallel robots (additionnally, some joints are not actuated, which causes additional problems).

The kinematic structure of a paralel robot indeed induces loops in the joint connection graph. In Pinocchio, we can only represents (one of) the underlying kinematic tree. The loop constraints have to be handled separately. An example that loads 4 manipulator arms is given below.

In [30]:
from utils. load_ur5_parallel import load_ur5_parallel  # noqa: E402

robot = load_ur5_parallel()

In [31]:
viz = MeshcatVisualizer(robot)
viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/


In [32]:
viz.display(robot.q0)

In [33]:
[w, h, d] = [0.5, 0.5, 0.005]
color = [red, green, blue, transparency] = [1, 1, 0.78, .8]
viz.addBox('world/robot0/toolplate', [w, h, d], color)
Mtool = pin.SE3(pin.utils.rotate('z', 1.268), np.array([0, 0, .75]))
viz.applyConfiguration('world/robot0/toolplate', Mtool)

The 4 legs of the robot are loaded in a single robot model. The 4 effector placements are computed by:

In [34]:
effIdxs = [robot.model.getFrameId('tool0_#%d' % i) for i in range(4)]
robot.framePlacement(robot.q0, effIdxs[0])

  R =
   -0.707107    -0.707107 -3.46245e-12
    0.707107    -0.707107 -3.46236e-12
-6.12323e-17 -4.89664e-12            1
  p = 0.306122 0.160483 0.749342

The loop constraints are that the relative placement of every leg end-effector must stay the same that in the initial configuration given as example in with the configuration *robot.q0* and the plate placement *Mtool*. To be valid, a configuration *q* must satisfy these 4 relative placement constraints.

Consider now that the orientation of the tool plate is given by the following quaternion, with the translation that you like (see [the notebook about rotations if you need more details](appendix1_quaternions.ipynb)): 



In [35]:
quat = pin.Quaternion(0.7, 0.2, 0.2, 0.6).normalized()
print(quat.matrix())

[[ 0.13978495 -0.8172043   0.55913978]
 [ 0.98924731  0.13978495 -0.04301075]
 [-0.04301075  0.55913978  0.82795699]]


- **Apply the rotation defined using the previous quaternion on the plate**
- **Find using the above optimization routines the configuration of each robot leg so that the loop constraints are all met** for the new orientation of the plate

In [36]:
# YOUR CODE